<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**imports**

In [18]:
# used some sniped from https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


**initial data+model setting**

In [19]:
batch_size = 128
dataset = MNIST('./',download=True, transform = transforms.Compose([ transforms.ToTensor(),transforms.Normalize([0.5], [0.5])]))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
image_size=784
hidden_size=128
lv_size = 64  #Latent Variable 
learning_rate=1e-4
criterion = nn.MSELoss()
num_epochs=200
n=1

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

**build model**

In [20]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, image_size),nn.ReLU(True), nn.Linear(image_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**train model**

In [23]:
model = autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

image_count=0
for epoch in range(num_epochs):
    for data in dataloader:
        img, lebal = data
        print('lebal'+str(lebal))
        img = img.view(img.size(0), -1)
        img = Variable(img)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    if 0: #epoch % 10 == 0:
        image_count+=1
        pic = to_img(output.cpu().data)
        nn,_,_,_ = np.shape(pic)
        print(np.shape(pic))
        for i in range(nn):
          ax = plt.subplot(2, n, image_count+i + 1)
          plt.imshow(pic[i,0,:,:].rehspae([28,28]))
          plt.title( "reconstruct image " )
          plt.gray()

torch.save(model.state_dict(), './sim_autoencoder.pth')

lebaltensor([9, 1, 0, 0, 0, 5, 0, 1, 7, 0, 7, 8, 0, 7, 0, 9, 9, 6, 7, 7, 5, 5, 2, 3,
        2, 9, 5, 5, 3, 9, 7, 6, 6, 2, 0, 2, 7, 2, 1, 5, 0, 4, 9, 5, 7, 2, 0, 9,
        0, 0, 5, 2, 3, 0, 4, 8, 3, 3, 8, 6, 6, 0, 8, 8, 4, 6, 6, 7, 4, 5, 3, 9,
        3, 2, 3, 3, 1, 2, 7, 4, 4, 6, 3, 3, 7, 7, 1, 5, 7, 9, 2, 0, 6, 5, 3, 8,
        2, 5, 5, 2, 2, 5, 8, 3, 0, 5, 5, 3, 0, 8, 2, 7, 6, 2, 7, 7, 0, 6, 3, 4,
        9, 0, 5, 3, 6, 8, 9, 7])
lebaltensor([3, 2, 6, 8, 9, 3, 5, 2, 7, 9, 8, 2, 2, 9, 2, 2, 3, 7, 3, 5, 8, 3, 3, 2,
        2, 7, 2, 3, 3, 0, 5, 3, 4, 8, 7, 9, 7, 6, 3, 0, 6, 8, 9, 9, 0, 7, 7, 0,
        5, 7, 2, 2, 8, 9, 6, 6, 8, 1, 4, 1, 8, 4, 1, 8, 6, 3, 5, 4, 1, 7, 8, 7,
        3, 5, 5, 6, 5, 5, 8, 0, 2, 1, 1, 5, 4, 3, 7, 5, 3, 1, 5, 1, 3, 7, 3, 8,
        8, 2, 3, 7, 6, 3, 8, 2, 4, 1, 6, 2, 8, 1, 0, 2, 1, 3, 6, 0, 1, 2, 5, 6,
        6, 3, 0, 7, 5, 9, 0, 6])
lebaltensor([8, 2, 4, 4, 8, 9, 5, 2, 7, 0, 6, 6, 2, 7, 8, 3, 1, 1, 0, 3, 9, 2, 6, 4,
        0, 6, 9, 4, 2, 1, 8, 6, 4, 1, 8

KeyboardInterrupt: ignored